In [13]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup

In [14]:
# Load the dictionary from the pickle file
with open('mydict_dict.pickle', 'rb') as file:
    mydict = pickle.load(file)

In [15]:
with open('mustra_p.html', mode='r', encoding='utf-8') as m:
    mustra = m.read()
    
soup = BeautifulSoup(mustra , 'lxml')

In [16]:
for key in mydict.keys():

    # týždne sú oddelené hrubšími čiarami, jednotlivé týždne sú zabalené v <tbody> elementoch
    current_date = pd.to_datetime(key)
    if (key == '2023-01-01') or current_date.day_name('sk_SK')=='Pondelok':
        tbody_here = soup.new_tag('tbody') # create new tbody here
        soup.table.append(tbody_here)

    tr = soup.new_tag('tr')
    # tr['id'] = key # id='2023-01-01'

    velkost = len(mydict[key]) # length of nested dictionary

    for variable in mydict[key].keys(): # prechádzame nested dictionary
        if variable == 'meniny': # pole "meniny"
            td = soup.new_tag('td')
            td['id'] = key # id='2023-01-01'
            td['class']='meniny' # na túto class je napojené CSS, aby sa nezalamoval m_string.

            # meniny string obohatený o ďalšie údaje o dni, this will be crucial
            meniny = mydict[key][variable]
            m_string = (
                str(current_date.day)+'. '+current_date.month_name('sk_SK').lower()+'\n'+current_date.day_name('sk_SK')+'\n<small><i>'+meniny+'</i></small>'
            )
            app = BeautifulSoup(m_string,'html.parser')
            td.append(app)

            if velkost>2:
                td['rowspan']=str(velkost-1)      

            tr.append(td)

        else: # zvyšné polia (0,1,2)
            td = soup.new_tag('td')
            td['class']='middle'
            td.string = mydict[key][variable][0] # v liturgii... nultý (prvý) údaj v liste
            tr.append(td)

            td = soup.new_tag('td')
            td.string = mydict[key][variable][1] # poznámka... prvý (druhý) údaj v liste
            tr.append(td)

            if velkost>2: # ak je v dni je viac možností, musíme pridať ďalší riadok (row)
                tbody_here.append(tr)
                tr = soup.new_tag('tr')

    tbody_here.append(tr)

In [17]:
# soup

In [18]:
with open('onepage_plain.html', mode='w', encoding='utf8') as f:
    f.write(str(soup))

In [8]:
# done.